In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#To extract the rar file

!pip install RarFile

from rarfile import RarFile

with RarFile("/content/drive/MyDrive/sketchy_dataset/10Classes.rar") as rf:
    rf.extractall()

# Reading the dataset

In [ ]:
import tensorflow as tf

In [ ]:
completeData = tf.keras.preprocessing.image_dataset_from_directory('/content/10Classes', 
                                                                    labels='inferred', 
                                                                    label_mode='categorical',
                                                                    batch_size=32,
                                                                    image_size=(224, 224), 
                                                                    seed=3)


Found 5672 files belonging to 10 classes.


In [ ]:
import numpy as np

X = []
Y = []


for td in completeData:
  for tx in td[0]:
    X.append(tx)
  for ty in td[1]:
    Y.append(ty)

X = np.array(X)
Y = np.array(Y)


print(X.shape, Y.shape)

(5672, 224, 224, 3) (5672, 10)


# Extracting features from images using pretrained VGG16 model

In [ ]:
from keras.applications.vgg16 import VGG16 
from keras.models import Model

model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

features = model.predict(X, use_multiprocessing=True)

553467904/553467096 [==============================] - 3s 0us/step


In [ ]:
features.shape, Y.shape

((5672, 4096), (5672, 10))

# Reducing Dimensionality of the Data set

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100, random_state=22)
pca.fit(features)
x = pca.transform(features)

In [ ]:
x.shape, Y.shape

((5672, 100), (5672, 10))

# Standarizing the dataset 

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(x)
x_std = sc.transform(x)

# Splitting Dataset into train and test set

In [ ]:
import sklearn.model_selection as model_selection

train_x, test_x, train_y, test_y = model_selection.train_test_split(x_std, Y, shuffle=True, test_size=0.2, random_state = 3)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(4537, 100) (4537, 10) (1135, 100) (1135, 10)


# Using RandomForestClassifier over our dataset

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 7)  
  
clf.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=7,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# performing predictions on the test dataset
y_pred = clf.predict(test_x)
  
# metrics are used to find accuracy or error
from sklearn import metrics  
print()
  
# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(test_y, y_pred))


ACCURACY OF THE MODEL:  0.5427312775330396


# Using Naive Bayes over our dataset

In [ ]:
train_x.shape, train_y.shape

((4537, 100), (4537, 10))

In [ ]:
labels = np.argmax(train_y, axis=1)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn import model_selection

In [ ]:
gnb = GaussianNB()

scores = model_selection.cross_val_score(gnb, train_x, labels, cv=5, scoring='accuracy')

print("Mean Score", scores.mean())

Mean Score 0.7954628464852421


# Using SVM classifier over the given dataset

In [ ]:
from sklearn.svm import SVC

# Training a SVM classifier using SVC class
svm = SVC(kernel= 'rbf', random_state=1, C=0.1)
svm.fit(train_x, labels)
 
# Mode performance
y_labels = np.argmax(test_y, axis=1)
y_pred = svm.predict(test_x)
print('Accuracy: %.3f' % accuracy_score(y_labels, y_pred))

Accuracy: 0.849


# Now trying the models on custom images

In [ ]:
import cv2 

def read_image(file):
  originalImage = cv2.imread(file)
  img = cv2.resize(originalImage, (224,224), interpolation=cv2.INTER_AREA)
  img = np.array([img])
  print(img.shape)
  return img

In [ ]:
def return_reduced_features(model_vgg, img):
  feat = model_vgg.predict(img)
  x = pca.transform(feat)
  return x

In [ ]:
def predict_model(model, x):
  pred = model.predict(x)
  return np.argmax(pred)

In [ ]:
def get_label(model, image_path, model_vgg):
  img = read_image(image_path)
  x = return_reduced_features(model_vgg, img)
  pred = predict_model(model, x)
  return pred

# Now let's use a Neural Network to do all of this but with more number of classes

In [ ]:
with RarFile("/content/drive/MyDrive/sketchy_dataset/Small.rar") as rf:
    rf.extractall()

In [ ]:
import tensorflow as tf

bigDataset = tf.keras.preprocessing.image_dataset_from_directory('/content/Small', 
                                                                    labels='inferred', 
                                                                    label_mode='categorical',
                                                                    batch_size=32,
                                                                    image_size=(224, 224), 
                                                                    validation_split=0.5,   
                                                                    subset='training',
                                                                    seed=3)

Found 13949 files belonging to 24 classes.
Using 6975 files for training.


In [ ]:
import numpy as np

X = []
Y = []


for td in bigDataset:
  for tx in td[0]:
    X.append(tx)
  for ty in td[1]:
    Y.append(ty)

X = np.array(X)
Y = np.array(Y)


print(X.shape, Y.shape)

(6975, 224, 224, 3) (6975, 24)


In [ ]:
from keras.applications.vgg16 import VGG16 
from keras.models import Model

model_vgg = VGG16()
model_vgg = Model(inputs = model_vgg.inputs, outputs = model_vgg.layers[-2].output)

features = model_vgg.predict(X, use_multiprocessing=True)

In [ ]:
import sklearn.model_selection as model_selection

train_x, test_x, train_y, test_y = model_selection.train_test_split(features, Y, shuffle=True, test_size=0.2, random_state = 3)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(5580, 4096) (5580, 24) (1395, 4096) (1395, 24)


In [ ]:
from tensorflow.keras import layers
from tensorflow import keras 

# Creating a Simple Dense Network 

model = keras.Sequential()
model.add(layers.Dense(1024, activation='relu', input_shape=train_x.shape[1:]))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(24, activation='softmax')) 
# Train model
adam = keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['top_k_categorical_accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              4195328   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                3

In [ ]:
model.fit(train_x, train_y, validation_split=0.1, batch_size = 256, verbose=2, epochs=5)

Epoch 1/5
20/20 - 1s - loss: 2.2912 - top_k_categorical_accuracy: 0.6780 - val_loss: 1.2248 - val_top_k_categorical_accuracy: 0.9158
Epoch 2/5
20/20 - 0s - loss: 1.2343 - top_k_categorical_accuracy: 0.9072 - val_loss: 0.9307 - val_top_k_categorical_accuracy: 0.9391
Epoch 3/5
20/20 - 0s - loss: 0.9609 - top_k_categorical_accuracy: 0.9401 - val_loss: 0.8174 - val_top_k_categorical_accuracy: 0.9444
Epoch 4/5
20/20 - 0s - loss: 0.7810 - top_k_categorical_accuracy: 0.9598 - val_loss: 0.7538 - val_top_k_categorical_accuracy: 0.9624
Epoch 5/5
20/20 - 0s - loss: 0.6631 - top_k_categorical_accuracy: 0.9729 - val_loss: 0.6811 - val_top_k_categorical_accuracy: 0.9624


In [ ]:
score = model.evaluate(test_x, test_y, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

Test accuarcy: 96.77%


# Predicting on custom image using this nn

In [ ]:
import cv2 
 
originalImage = cv2.imread('star.png')
img = cv2.resize(originalImage, (224,224), interpolation=cv2.INTER_AREA)
img = np.array([img])
print(img.shape)

(1, 224, 224, 3)


In [ ]:
feat = model_vgg.predict(img)

pred = model.predict(feat)
np.argmax(pred)

14